In [10]:
import fonctions


In [17]:
from math import *
rho = 6.371*10**6
def convertirLat(phi1, phi2):
    return rho*2*pi/360*(phi1-phi2)

def convertirLon(phi1, phi2):
    return rho*cos(2*pi*2.34842/360)*2*pi/360*(phi1-phi2)



In [11]:
import pandas as pd
instance = pd.read_csv("instances/instance_01.csv")

vehicules = pd.read_csv("vehicles.csv")
def reference_travel_time(f, i, j):
    x1 = instance.loc[i]["longitude"]
    x2 = instance.loc[j]["longitude"]
    y1 = instance.loc[i]["latitude"]
    y2 = instance.loc[j]["latitude"]
    sf = vehicules.loc[f]["speed"]
    pf = vehicules.loc[f]["parking_time"]
    return fonctions.distanceE((x1,y1), (x2,y2))/sf + pf

In [20]:
def miam(f,t):
    k = 0 
    a = [vehicules.loc[f]["fourier_cos_0"],vehicules.loc[f]["fourier_cos_1"],vehicules.loc[f]["fourier_cos_2"],vehicules.loc[f]["fourier_cos_3"]]
    b = [vehicules.loc[f]["fourier_sin_0"],vehicules.loc[f]["fourier_sin_1"],vehicules.loc[f]["fourier_sin_2"],vehicules.loc[f]["fourier_sin_3"]]
    for i in range(4):
        k+= a[i]*cos(i*2*pi/86400*t) + b[i]*sin(i*2*pi/86400 * t)
    return k

In [19]:
def travel_time(f, i, j, t):
    return reference_travel_time(f, i, j)*miam(f,t)

In [26]:
def rental_cost(route):
    return vehicules.loc[route[0]]["rental_cost"]


In [27]:
rental_cost([1,2])

390.0

In [29]:
def fuel_cost(route):
    cout = vehicules.loc[route[0]]["fuel_cost"]
    chemin = route[2]
    k=0
    for i in range(len(chemin)-1):
        x1 = instance.loc[i]["longitude"]
        x2 = instance.loc[i+1]["longitude"]
        y1 = instance.loc[i]["latitude"]
        y2 = instance.loc[i+1]["latitude"]
        k+=fonctions.distanceM((x1,y1),(x2,y2))
    
    return cout*k


In [31]:
def radius_cost(route):
    cout = vehicules.loc[route[0]]["radius_cost"]
    chemin = route[2]
    maxi = 0
    for i in range(1,len(chemin)-1):
        for j in range(1, len(chemin)-1):
            x1 = instance.loc[i]["longitude"]
            x2 = instance.loc[j]["longitude"]
            y1 = instance.loc[i]["latitude"]
            y2 = instance.loc[j]["latitude"]
            dist=fonctions.distanceE((x1,y1),(x2,y2))
            if dist>maxi:
                maxi = dist

    return cout*(0.5*maxi)**2

In [33]:
def cout_total(sol):
    total = 0 
    for i in sol:
        total += rental_cost(i)+ fuel_cost(i) + radius_cost(i)
    return total